# Семантика

## Завдання

Одне із змагань на SemEval-2015 було пов'язане з пошуком перефразувань та схожих за змістом твітів - [Paraphrase and Semantic Similarity in Twitter](http://alt.qcri.org/semeval2015/task1/). Ваше завдання цього тижня - побудувати модель, яка буде визначати перефразування та/або семантичну подібність краще, ніж бейзлайн у цьому змаганні.

Побудуйте одне із запропонованих рішень для цієї задачі (на вибір):

1. Побудуйте класифікатор на лінгвістичних ознаках (сутності, енграми слів чи символів, корені слів, редакторська відстань, синтаксична та семантична подібність тощо). **Обов'язкова умова** - використання семантичних ознак. Наприклад:
   - перетин слів та їх синонімів, гіпонімів, пов'язаних за темою слів тощо
   - середня близькість речень по онтології
   - редакторська відстань по AMR-графу
   - збіги по семантичних ролях

NE overlap( people, dates, currency), unigram/bigram overlap, stem overlap
сер близбкість речень по онтології (буз служ чатсин мови)
t1 
t2
найкоротшиа відстань 

2. Побудуйте класифікатор на основі ваших улюблений векторних представлень слів. Поекспериментуйте з різними способами агрегації, відбору та зважування векторів. **Обов'язкова умова** - використання семантично збагачених векторів (як основне рішення чи для порівняння). Можна взяти готові вектори (наприклад, [ConceptNet Numberbatch](https://github.com/commonsense/conceptnet-numberbatch)) або самостійно виконати [ретрофіттинг](https://github.com/mfaruqui/retrofitting/blob/master/retrofit.py) на будь-яких стандартних векторах.

Більше натхнення щодо ознак та варіантів побудови рішення можна черпати зі [звіту про змагання](https://www.aclweb.org/anthology/S15-2001).

### Дані

Використайте дані та метрики з репозиторію змагання SemEval-2015 (архів опубліковано через Слек). Вся інформація щодо формату корпусу та метрик міститься в Readme.md.

Запустити метрику можна таким способом:
```
$ python scripts/pit2015_eval_single.py data/test.label systemoutputs/PIT2015_BASELINE_02_LG.output
838     BASELINE        02_LG           F: 0.589        Prec: 0.679     Rec: 0.520              P-corr: 0.511   F1: 0.601       Prec: 0.674     Rec: 0.543
```

**Важливо**: не заточуйтесь на test-дані. У корпусі є окремо виділений dev-сет, на якому можна порівнювати ваші рішення.






# 1. Given

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import porter

from _pickle import load
from _pickle import dump

from collections import *

In [327]:
import re
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

# import nltk
# from langdetect import detect 
# from sklearn.model_selection import train_test_split
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import preprocessing
# from sklearn.metrics import precision_recall_fscore_support
# from sklearn.metrics import classification_report
import pickle

## 1.1 Read Data

In [70]:
# read from train/test data files and create features
def readInData(filename):

    data = []
    trends = set([])
    
    (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = (None, None, None, None, None, None, None)
    
    check = []
    for i, line in enumerate(open(filename)):
        line = line.strip()
        #read in training or dev data with labels
        if len(line.split('\t')) == 7:
            (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = line.split('\t')
            check.append((i, 7))
        #read in test data without labels
        elif len(line.split('\t')) == 6:
            (trendid, trendname, origsent, candsent, origsenttag, candsenttag) = line.split('\t')
            check.append((i, 6))
        else:
            check.append((i, 0))
            continue
        
        #if origsent == candsent:
        #    continue
        
        trends.add(trendid)
        #features = paraphrase_Das_features(origsent, candsent, trendname)
        
        if judge == None:
            data.append((judge, origsent, candsent,  origsenttag, candsenttag, trendid))
            continue

        # ignoring the training/test data that has middle label 
        if judge[0] == '(':  # labelled by Amazon Mechanical Turk in format like "(2,3)"
            nYes = eval(judge)[0]            
            if nYes >= 3:
                amt_label = True
                data.append((amt_label, origsent, candsent,  origsenttag, candsenttag, trendid))
            elif nYes <= 1:
                amt_label = False
                data.append((amt_label, origsent, candsent,  origsenttag, candsenttag, trendid))   
        elif judge[0].isdigit():   # labelled by expert in format like "2"
            nYes = int(judge[0])
            if nYes >= 4:
                expert_label = True
                data.append((expert_label, origsent, candsent,  origsenttag, candsenttag, trendid))
            elif nYes <= 2:
                expert_label = False
                data.append((expert_label, origsent, candsent,  origsenttag, candsenttag, trendid))     
            else:
            	expert_label = None
            	data.append((expert_label, origsent, candsent,  origsenttag, candsenttag, trendid))        
                
    return pd.DataFrame(data, columns =['label', 'origsent', 'candsent',  'origsenttag', 'candsenttag', 'trendid']),\
            trends, check
    



In [47]:
trainfilename = "data/train.data"
testfilename  = "data/test.data"
# modelfile = 'scripts//baseline_logisticregression.model'

In [6]:
trainfull, traintrends  = readInData(trainfilename)    

In [216]:
testfull, testtrends, check  = readInData(testfilename)

In [75]:
testfull.head()

,label,origsent,candsent,origsenttag,candsenttag,trendid
0,None,All the home alones watching 8 mile,8 mile is on thats my movie,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP/O alones/O/VBZ/B-VP/O watching/O/VBG/I-VP/B-EVENT 8/O/CD/B-NP/O mile/O/NN/I-NP/O,8/O/NN/B-NP/O mile/O/NN/I-NP/O is/O/VBZ/B-VP/O on/O/IN/B-PP/O thats/O/NNS/B-NP/O my/O/PRP$/B-NP/O movie/O/NN/I-NP/B-EVENT,51
1,False,All the home alones watching 8 mile,The last rap battle in 8 Mile nevr gets old ahah,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP/O alones/O/VBZ/B-VP/O watching/O/VBG/I-VP/B-EVENT 8/O/CD/B-NP/O mile/O/NN/I-NP/O,The/O/DT/B-NP/O last/O/JJ/I-NP/O rap/O/NN/I-NP/B-EVENT battle/O/NN/I-NP/B-EVENT in/O/IN/B-PP/O 8/O/CD/B-NP/O Mile/O/NNP/I-NP/O nevr/O/NN/I-NP/O gets/O/VBZ/B-VP/O old/O/JJ/B-NP/O ahah/O/JJ/I-NP/O,51
2,False,All the home alones watching 8 mile,The rap battle at the end of 8 mile gets me so hype,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP/O alones/O/VBZ/B-VP/O watching/O/VBG/I-VP/B-EVENT 8/O/CD/B-NP/O mile/O/NN/I-NP/O,The/O/DT/B-NP/O rap/O/NN/I-NP/O battle/O/NN/I-NP/B-EVENT at/O/IN/B-PP/O the/O/DT/B-NP/O end/O/NN/I-NP/O of/O/IN/B-PP/O 8/O/CD/B-NP/O mile/O/NN/I-NP/O gets/O/VBZ/B-VP/O me/O/PRP/B-NP/O so/O/RB/B-ADVP/O hype/O/JJ/I-ADVP/O,51
3,False,The Ending to 8 Mile is my fav part of the whole movie,Rabbit on 8 mile out of place but determined to make it,The/O/DT/B-NP/O Ending/O/VBG/I-NP/B-EVENT to/O/TO/I-NP/O 8/O/CD/I-NP/O Mile/O/NNP/I-NP/O is/O/VBZ/B-VP/O my/O/PRP$/B-NP/O fav/O/JJ/I-NP/O part/O/NN/I-NP/O of/O/IN/B-PP/O the/O/DT/B-NP/O whole/O/JJ/I-NP/O movie/O/NN/I-NP/B-EVENT,Rabbit/O/NNP/B-NP/O on/O/IN/B-PP/O 8/O/CD/B-NP/O mile/O/NN/I-NP/O out/O/IN/B-PP/O of/O/IN/B-PP/O place/O/NN/B-NP/O but/O/CC/O/O determined/O/VBD/B-VP/B-EVENT to/O/TO/I-VP/O make/O/VB/I-VP/O it/O/PRP/B-NP/O,51
4,False,The Ending to 8 Mile is my fav part of the whole movie,See 8 Mile is always on but it s the tv version so it s gay,The/O/DT/B-NP/O Ending/O/VBG/I-NP/B-EVENT to/O/TO/I-NP/O 8/O/CD/I-NP/O Mile/O/NNP/I-NP/O is/O/VBZ/B-VP/O my/O/PRP$/B-NP/O fav/O/JJ/I-NP/O part/O/NN/I-NP/O of/O/IN/B-PP/O the/O/DT/B-NP/O whole/O/JJ/I-NP/O movie/O/NN/I-NP/B-EVENT,See/O/VB/B-VP/O 8/O/CD/B-NP/O Mile/O/NNP/I-NP/O is/O/VBZ/B-VP/O always/O/RB/B-ADVP/O on/O/IN/B-PP/O but/O/CC/O/O it/O/PRP/B-NP/O s/O/VBZ/B-VP/O the/O/DT/B-NP/O tv/O/NN/I-NP/O version/O/NN/I-NP/B-EVENT so/O/IN/B-SBAR/O it/O/PRP/B-NP/O s/O/RB/B-ADJP/O gay/O/JJ/I-ADJP/O,51


In [8]:
pd.set_option('display.max_colwidth', 500)

In [78]:
model_vectors = KeyedVectors.load_word2vec_format("word_vectors/numberbatch-en-17.06.txt.gz", binary=False)

In [ ]:
# E.g.:
# model.most_similar(positive=['breakfast'], topn=10)

In [278]:
def getTokens(senttag):
    return [i.split('/')[0].lower() for i in senttag.split()]

In [385]:
def getTextVect(senttag):
    tokens = getTokens(senttag)
    vectors = np.array([model_vectors.get_vector(i) for i in tokens\
                        if (i in model_vectors.vocab)
                           #&
                           #(i not in set(stopwords.words('english')))\
                       ])  
    vec = np.sum(vectors, axis=0)/len(vectors)
    #vec = np.min(vectors, axis=0)/len(vectors)
    if vec.size != 0:
        return vec
    else:
        return np.zeros(300)

def givenConcat(row):

    return np.append(\
            np.concatenate((row['origDocVec'], row['candDocVec']))
            ,
            (row['wmdistance'] + row['cosineSim']))
            

In [386]:
def getCosSim(row):
    return cosine_similarity(row['origDocVec'].reshape(1,300), row['candDocVec'].reshape(1,300))

In [387]:
def getFeatures(trainfull):  
    trainfull['origDocVec'] = trainfull['origsenttag'].map(getTextVect)
    trainfull['candDocVec'] = trainfull['candsenttag'].map(getTextVect)
    trainfull['wmdistance'] = trainfull.apply(lambda row: model_vectors.wmdistance(getTokens(row['origsenttag']),\
                                                               getTokens(row['candsenttag'])),
                        axis=1 )
    
    trainfull['cosineSim'] = trainfull.apply(getCosSim, axis=1)
    
#     return trainfull
    return trainfull[['wmdistance', 'origDocVec', 'candDocVec', 'cosineSim']].apply(givenConcat, axis=1)

In [388]:
X_train = getFeatures(trainfull)
y_train = trainfull['label']

In [389]:
lrc = LogisticRegression(random_state=42, solver="lbfgs")
lrc.fit(list(X_train),  list(y_train.values.ravel()))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

**Load test set**

In [390]:
testfull['features'] = getFeatures(testfull)

In [391]:
testfull_1 = testfull[testfull['label'].notna()]

In [394]:
X_test = testfull_1[['features']]
y_test = testfull_1['label']

In [395]:
predicted = lrc.predict(list(X_test['features']))

In [396]:
print(classification_report(y_true=list(y_test), y_pred=predicted))

              precision    recall  f1-score   support

       False       0.83      0.96      0.89       663
        True       0.61      0.25      0.35       175

   micro avg       0.81      0.81      0.81       838
   macro avg       0.72      0.60      0.62       838
weighted avg       0.78      0.81      0.78       838



In [397]:
predicted_proba = lrc.predict_proba(list(testfull['features']))

# Evaluation

In [398]:
# Load the trained model and output the predictions
def OutputPredictions(predicted_proba, outfile):
           
    # output the results into a file
    outf = open(outfile,'w') 
          
    for prob in predicted_proba:
        if prob[0] >= 0.5:
             outf.write("true\t" + "{0:.4f}".format(prob[0]) + "\n")
        else:
             outf.write("false\t" + "{0:.4f}".format(prob[0]) + "\n")
             
    outf.close()

In [399]:
outputfilename = "./systemoutputs/PIT2015_BASELINE_MY_LG.output"

In [400]:
OutputPredictions(predicted_proba, outputfilename)

In [401]:
!python scripts/pit2015_eval_single.py ./data/test.label ./systemoutputs/PIT2015_BASELINE_MY_LG.output

838	BASELINE	MY_LG		F: 0.280	Prec: 0.172	Rec: 0.754		P-corr: -0.381	F1: 0.346	Prec: 0.209	Rec: 1.000
